In [1]:
from xtagger import HiddenMarkovModel
from xtagger import EnglishRegExTagger
from xtagger import xMetrics

import nltk
import nltk
from sklearn.model_selection import train_test_split

nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
train_set, test_set = train_test_split(nltk_data,train_size=0.8,test_size=0.2)

In [2]:
model = HiddenMarkovModel(
    extend_to = "bigram",
    language = "en"
)

model.fit(train_set)

In [8]:
class MatthewsCorrCoef(xMetrics):
    def __init__(self, y_true, y_pred, tags):
        super(MatthewsCorrCoef, self).__init__(y_true, y_pred, tags)
        
    def __call__(self):
        import numpy as np
        from sklearn.metrics import confusion_matrix
        C = confusion_matrix(self.y_true.argmax(axis=1),self. y_pred.argmax(axis=1))
        
        t_sum = C.sum(axis = 1, dtype = np.float64)
        p_sum = C.sum(axis = 0, dtype = np.float64)
        n_correct = np.trace(C, dtype = np.float64)
        n_samples = p_sum.sum()
        cov_ytyp = n_correct * n_samples - np.dot(t_sum, p_sum)
        cov_ypyp = n_samples ** 2 - np.dot(p_sum, p_sum)
        cov_ytyt = n_samples ** 2 - np.dot(t_sum, t_sum)
        mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
        
        if np.isnan(mcc):
            return 0.
        else:
            return mcc

In [9]:
model.evaluate(
    test_set,
    random_size = 5,
    seed = 15,
    eval_metrics = ['acc', 'avg_recall', MatthewsCorrCoef],
    result_type = "%",
)

{'acc': 95.34883720930233,
 'avg_recall': {'weigted': 95.34883720930233,
  'micro': 95.34883720930233,
  'macro': 92.18657159833631},
 'MatthewsCorrCoef': 0.9469769507004047}